# HW 8

In [2]:
import pandas as pd
import numpy as np


In [3]:
# Import the BX-Book-Ratings file - specify that the delimiter is a semicolon, not a comma. 
bkratings = pd.read_csv("BX-Book-Ratings.csv", sep=';')
# Check import
bkratings.head()
bkratings.info()

# Content description:
# User ID, ISBN of book reviewed, book rating.

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1149780 entries, 0 to 1149779
Data columns (total 3 columns):
﻿User-ID       1149780 non-null int64
ISBN           1149780 non-null object
Book-Rating    1149780 non-null int64
dtypes: int64(2), object(1)
memory usage: 35.1+ MB


In [4]:
# Import the books file, use "," as the delimiter otherwise it doesn't import correctly.
bktitles = pd.read_csv("BX-Books.csv", sep='";"')
# Because of the delimiter, we have a leading quote. Rename the first column (ISBN)
bktitles.rename(columns={'"ISBN':'ISBN','Image-URL-L"':'Image-URL-L' }, inplace=True)
# Strip the leading quote from the ISBN column. 
bktitles['ISBN'] = bktitles['ISBN'].apply(lambda x:x.strip('"'))
# Check data import
bktitles.head()
bktitles.info()

# Content description:
# ISBN, book title, author, year of publication, publisher, book image URLs (3 sizes - small, med, large)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271379 entries, 0 to 271378
Data columns (total 8 columns):
ISBN                   271379 non-null object
Book-Title             271379 non-null object
Book-Author            271379 non-null object
Year-Of-Publication    271379 non-null int64
Publisher              271377 non-null object
Image-URL-S            271379 non-null object
Image-URL-M            271379 non-null object
Image-URL-L            271379 non-null object
dtypes: int64(1), object(7)
memory usage: 18.6+ MB


c:\Users\Admin\Anaconda\lib\site-packages\pandas\io\parsers.py:648: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators; you can avoid this warning by specifying engine='python'.
  ParserWarning)


In [5]:
# Zip the ISBN & book titles together into a dictionary
# # We do this because we want to be able to just reference the books - not have to look up books using ISBNs each time
# bkdict = dict(zip(bktitles['ISBN'], bktitles['Book-Title']))
# print bkdict

In [6]:
# Import the book review users file
bkauthors = pd.read_csv("BX-Users.csv", sep=';', quotechar='"')
bkauthors.head()

# Content description:
# User ID, user location, age


,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17
4,5,"farnborough, hants, united kingdom",NaN


In [7]:
# Create dataframe that joins the reviews & book information columns together, on the ISBN column
# Drop extra columns that we are not using
merged_bks = pd.merge(bkratings, bktitles, how='inner', on='ISBN')
merged_bks.info()

# In case we wanted to drop extraneous columns - code below. 
# .drop(['Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1)

# Check that dataframe is formatted correctly
merged_bks.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031175 entries, 0 to 1031174
Data columns (total 10 columns):
﻿User-ID               1031175 non-null int64
ISBN                   1031175 non-null object
Book-Rating            1031175 non-null int64
Book-Title             1031175 non-null object
Book-Author            1031175 non-null object
Year-Of-Publication    1031175 non-null int64
Publisher              1031173 non-null object
Image-URL-S            1031175 non-null object
Image-URL-M            1031175 non-null object
Image-URL-L            1031175 non-null object
dtypes: int64(3), object(7)
memory usage: 86.5+ MB


,﻿User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
2,6543,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
3,8680,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
4,10314,034545104X,9,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...


# Book recommendations

In [14]:
print merged_bks.columns

Index([u'﻿User-ID', u'ISBN', u'Book-Rating', u'Book-Title', u'Book-Author',
       u'Year-Of-Publication', u'Publisher', u'Image-URL-S', u'Image-URL-M',
       u'Image-URL-L'],
      dtype='object')


In [20]:
merged_bks['userid'] = merged_bks[merged_bks.columns[0]]
merged_bks.userid.head()

0    276725
1      2313
2      6543
3      8680
4     10314
Name: userid, dtype: int64

In [21]:
# Create pivot table 
df_pivot = pd.pivot_table(merged_bks, values=["Book-Rating"],
        columns=["userid","Book-Title"],
        aggfunc=np.mean)
# Major debugging needed on this.

#pivot_table converts to a multi-index series. Unstack converts to a dataframe where the last index becomes our column head
# df_wide = df_pivot.unstack(-1)
# df_wide

In [31]:
# Fill nulls with 0s
# df_wide = df_wide.fillna(0)

In [32]:
# Check that columns have user-IDs
# df_wide.columns[:10]

In [27]:
# Check that rows include book titles
# df_wide.index.levels[0]
# book_names = df_wide.index.levels[1]
# book_names

In [28]:
# Import libraries to calculate cosine similarity
# from sklearn.metrics.pairwise import cosine_similarity
# calculates cosine similarity of vectors
# from sklearn.metrics.pairwise import manhattan_distances
# from sklearn.metrics.pairwise import euclidean_distances

In [29]:
# Calculate distances
# dists = cosine_similarity(df_wide)
# dists

In [30]:
# Convert distances to a dataframe
# dists = pd.DataFrame(dists)
# dists.columns = book_names
# dists.index = book_names
# dists.ix[0:10, 0:10]

# dists

In [33]:
# Create function that allows users to enter their books of choice and the number of book suggestions they'd like
# def get_books(book_choices, x):
#     if type(book_choices) != list:
#         return "Please enter list:" 
#     else: 
#         books_summed = dists[book_choices].apply(lambda row: np.sum(row), axis=1)
#         ranked_books = books_summed.index[books_summed.index.isin(beer_choices)==False]
#         ranked_books = ranked_books.tolist()
#         return ranked_books[:x]

# User similarities

In [ ]:
# Create pivot table 
df_pivot = pd.pivot_table(merged_bks, values=["Book-Rating"],
        columns=["Book-Title", "User-ID"],
        aggfunc=np.mean)
# Major debugging needed on this.

#pivot_table converts to a multi-index series. Unstack converts to a dataframe where the last index becomes our column head
# df_wide = df_pivot.unstack(-1)
# df_wide

